# day 2-3

---

## 0. はじめに

ページ上部のメニューバーにある **Kernel** メニューをクリックし、プルダウンメニューから [**Change Kernel ...**] を選び、**gssm2023:Python** を選択してください。

<img src="images/change_kernel1.png" width="30%">

ノートブック上部の右隅に表示されたカーネル名が **gssm2023:Python** になっていることを確認してください。

<img src="images/change_kernel2.png" width="30%">

---

## 1. テキスト解析 (2)

### 1.0 事前準備 (関数の定義)

以下のセルを実行してください

In [ ]:
# ワードクラウドを描画する
def draw_wordcloud(word_counts_df):

    font_path = !find ${HOME} -name "ipaexg.ttf"

    import matplotlib.pyplot as plt
    import wordcloud
    %matplotlib inline

    wc = wordcloud.WordCloud(
        background_color='white',
        font_path=font_path[0],
        max_font_size=100)

    img = wc.generate(' '.join(word_counts_df['表層'].tolist()))
    plt.figure(figsize=(8, 4))
    plt.imshow(img, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()


# トピックモデルによるワードクラウドを描画する
def draw_topic_model(lda, feature_names, n_top_words=20):

    font_path = !find ${HOME} -name "ipaexg.ttf"

    import matplotlib.pyplot as plt
    import wordcloud
    %matplotlib inline

    fig = plt.figure(figsize=(10,4))

    for topic_idx, topic in enumerate(lda.components_):
        sorted_text = ' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]])

        wc = wordcloud.WordCloud(
            background_color='white',
            font_path=font_path[0],
            max_font_size=100)

        ax = fig.add_subplot(2, 3, topic_idx + 1)
        img = wc.generate(sorted_text)
        ax.imshow(img, interpolation='bilinear')
        ax.set_title(f"Topic # {topic_idx+1}:")

    plt.tight_layout()
    plt.show()


# 共起ネットワークを描画する
def plot_cooccur_network(df, word_counts, cutoff):

    import networkx as nx
    from networkx.algorithms import community
    import matplotlib.pyplot as plt
    import japanize_matplotlib
    from networkx.drawing.nx_agraph import graphviz_layout
    %matplotlib inline

    Xc = df.values
    Xc_max = Xc.max()

    words = df.columns
    count_max = word_counts.max()

    weights_w, weights_c = [], []
    for i, j in zip(*Xc.nonzero()):
        if i < j and Xc[i,j] > cutoff:
            weights_w.append((words[i], {'weight': word_counts[i] / count_max}))
            weights_w.append((words[j], {'weight': word_counts[j] / count_max}))
            weights_c.append((words[i], words[j], Xc[i,j] / Xc_max))

    G = nx.Graph()
    G.add_nodes_from(weights_w)
    G.add_weighted_edges_from(weights_c)
    G.remove_nodes_from(list(nx.isolates(G)))
    # G = nx.minimum_spanning_tree(G)

    plt.figure(figsize=(10, 10))
    # pos = nx.spring_layout(G, k=0.3)
    pos = graphviz_layout(G, prog='neato', args='-Goverlap="scalexy" -Gsep="+6" -Gnodesep=0.8 -Gsplines="polyline" -GpackMode="graph" -Gstart={}'.format(43))
    weights_n = np.array(list(nx.get_node_attributes(G, 'weight').values()))
    weights_e = np.array(list(nx.get_edge_attributes(G, 'weight').values()))

    communities = community.greedy_modularity_communities(G)
    color_map = []
    for node in G:
        for i, c in enumerate(communities):
            if node in c:
                color_map.append(i)

    nx.draw_networkx_nodes(G, pos, node_color=color_map, alpha=0.7, cmap=plt.cm.Set2, node_size=5000 * weights_n)
    nx.draw_networkx_edges(G, pos, edge_color='gray', edge_cmap=plt.cm.Blues, alpha=0.7, width=3 * weights_e)
    nx.draw_networkx_labels(G, pos, font_family='IPAexGothic')

    plt.axis("off")
    plt.show()


# 係り受けによる共起ネットワークを描画する
def plot_dependency_network(df, word_counts, cutoff):

    import networkx as nx
    from networkx.algorithms import community
    import matplotlib.pyplot as plt
    import japanize_matplotlib
    from networkx.drawing.nx_agraph import graphviz_layout
    %matplotlib inline

    Xc = df.values
    Xc_max = Xc.max()

    words = df.columns
    count_max = word_counts.max()

    weights_w, weights_c = [], []
    for i, j in zip(*Xc.nonzero()):
        if Xc[i,j] > cutoff:
            weights_w.append((words[i], {'weight': word_counts[i] / count_max}))
            weights_w.append((words[j], {'weight': word_counts[j] / count_max}))
            weights_c.append((words[i], words[j], Xc[i,j] / Xc_max))

    G = nx.DiGraph()
    G.add_nodes_from(weights_w)
    G.add_weighted_edges_from(weights_c)
    G.remove_nodes_from(list(nx.isolates(G)))
    # G = nx.minimum_spanning_tree(G)

    plt.figure(figsize=(10, 10))
    # pos = nx.spring_layout(G, k=0.3)
    pos = graphviz_layout(G, prog='neato', args='-Goverlap="scalexy" -Gsep="+6" -Gnodesep=0.8 -Gsplines="polyline" -GpackMode="graph" -Gstart={}'.format(43))
    weights_n = np.array(list(nx.get_node_attributes(G, 'weight').values()))
    weights_e = np.array(list(nx.get_edge_attributes(G, 'weight').values()))

    communities = community.greedy_modularity_communities(G)
    color_map = []
    for node in G:
        for i, c in enumerate(communities):
            if node in c:
                color_map.append(i)

    nx.draw_networkx_nodes(G, pos, node_color=color_map, alpha=0.7, cmap=plt.cm.Set2, node_size=5000 * weights_n)
    nx.draw_networkx_edges(G, pos, edge_color='gray', edge_cmap=plt.cm.Blues, alpha=0.7, width=3 * weights_e)
    nx.draw_networkx_labels(G, pos, font_family='IPAexGothic')

    plt.axis("off")
    plt.show()


# 対応分析の結果をプロットする
def plot_coresp(df):

    import numpy as np

    table_N = df.values
    row_sum = table_N.sum(axis=1)
    col_sum = table_N.sum(axis=0)
    n = df.values.sum()

    expected = np.outer(row_sum, col_sum) / n
    chisq = np.square(table_N - expected) / expected
    residuals = (table_N - expected) / np.sqrt(expected)

    # Standardized residuals
    residuals = residuals / np.sqrt(n)

    # Number of dimensions
    nb_axes = min(residuals.shape[0]-1, residuals.shape[1]-1)

    # Singular value decomposition
    U, s, V = np.linalg.svd(residuals, full_matrices=True)
    s = s[:nb_axes]
    u = U[:, :nb_axes]
    v = V.T[:, :nb_axes]

    # row mass
    row_mass = row_sum / n

    # row coord = sv * u /sqrt(row.mass)
    cc = (u * s)
    row_coord = cc / np.sqrt(row_mass)[:, np.newaxis]

    # col mass
    col_mass = col_sum / n

    # row coord = sv * v /sqrt(col.mass)
    cc = (v * s)
    col_coord = cc / np.sqrt(col_mass)[:, np.newaxis]

    # eige nvalue
    eige_nvalue = s ** 2

    # contribution rate 
    contribution_rate = 100 * eige_nvalue / sum(eige_nvalue)
    contribution_rate

    import matplotlib.pyplot as plt
    import numpy as np

    # Plot of rows
    plt.plot(row_coord[:, 0], row_coord[:, 1], 'o', color='red')
    for i, label in enumerate(df.index):
        plt.text(row_coord[i, 0], row_coord[i, 1], label, color='red', ha='left', va='bottom')

    # Plot of columns
    plt.plot(col_coord[:, 0], col_coord[:, 1], 'o', color='blue')
    for i, label in enumerate(df.columns):
        plt.text(col_coord[i, 0], col_coord[i, 1], label, color='blue', ha='left', va='bottom')

    plt.axvline(0, linestyle='dashed', color='black')
    plt.axhline(0, linestyle='dashed', color='black')
    plt.xlabel(f"Dim 1 ({contribution_rate[0]:.3f}%)")
    plt.ylabel(f"Dim 2 ({contribution_rate[1]:.3f}%)")
    plt.show()


# 共起頻度行列を Jaccard 係数行列に変換する
def jaccard_coef(cooccur_df):

    Xc = cooccur_df.values
    Xj = np.zeros(Xc.shape)
    Xc_sum = cross_df.sum(axis=0).values

    for i, j in zip(*Xc.nonzero()):
        if i < j:
            Xj[i,j] = Xc[i,j] / (Xc[i,j] + Xc_sum[i] + Xc_sum[j])

    jaccard_df = pd.DataFrame(Xj, columns=cooccur_df.columns, index=cooccur_df.columns)

    return jaccard_df

### 1.1 データのダウンロード (前回ダウンロード済みのためスキップ)

以下のデータがダウンロード済みです

| ファイル名 | 件数 | データセット | 備考 |
| --- | --- | --- | --- |
| rakuten-1000-2022-2023.xlsx.zip | 10,000 | •レジャー+ビジネスの 10エリア<br>•エリアごと 1,000件 (ランダムサンプリング)<br>•期間: 2022/1~2023 GW明け | 本講義の全体を通して使用する |

In [ ]:
# もし、再度ダウンロードが必要な場合は残りの行のコメントマーク「#」を除去して、このセルを再実行してください

# FILE_ID = "1n-uvGoH7XQhxexN57hYXuFrkGeHKp-HV"
# !gdown --id {FILE_ID}
# !unzip rakuten-1000-2022-2023.xlsx.zip

### 1.2 データの読み込み (DataFrame型)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_excel("rakuten-1000-2022-2023.xlsx")
print(df.shape)
display(df.head())

### 1.3 単語の抽出

コメント列から単語を抽出する (単語を品詞「名詞」「形容詞」「未知語」で絞り込む)

In [ ]:
from collections import defaultdict
import MeCab

tagger = MeCab.Tagger("-r ../tools/usr/local/etc/mecabrc --unk-feature 未知語")

word_counts = defaultdict(lambda: 1)
words = []

ZEN = "".join(chr(0xff01 + i) for i in range(94))
HAN = "".join(chr(0x21 + i) for i in range(94))
HAN2ZEN = str.maketrans(HAN, ZEN)

# stopwords = ['する', 'ある', 'ない', 'いう', 'もの', 'こと', 'よう', 'なる', 'ほう']
stopwords = ["湯畑"]

for index, row in df.iterrows():
    node = tagger.parseToNode(row["コメント"].translate(HAN2ZEN))
    while node:
        features = node.feature.split(',')
        pos1 = features[0]
        pos2 = features[1] if len(features) > 1 else ""
        base = features[6] if len(features) > 6 else None

        if base not in stopwords:

            if (pos1 == "名詞" and pos2 == "一般"):
                base = base if base is not None else node.surface
                postag = "名詞"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            elif (pos1 == "名詞" and pos2 == "形容動詞語幹"):
                base = base if base is not None else node.surface
                base = f"{base}だ"
                postag = "形容動詞"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            elif pos1 == "形容詞":
                base = base if base is not None else node.surface
                postag = "形容詞"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            elif pos1 == "未知語":
                base = base if base is not None else node.surface
                postag = "未知語"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

        node = node.next

columns = [
    "文書ID",
    # "単語ID",
    "表層",
    "品詞",
    "カテゴリー",
    "エリア",
    "dict_key",
]
docs_df = pd.DataFrame(words, columns=columns)
print(docs_df.shape)
display(docs_df.head())

### 1.4 単語の出現頻度

単語の出現頻度をカウントする

In [ ]:
word_list = []
for i, (k, v) in enumerate(sorted(word_counts.items(), key=lambda x:x[1], reverse=True)):
    word_list.append((i, k[0], v, k))

columns = [
    "単語ID",
    "表層",
    "出現頻度",
    "dict_key"
]
word_counts_df = pd.DataFrame(word_list, columns=columns)[0:75]
print(word_counts_df.shape)
display(word_counts_df.head(10))

In [ ]:
merged_df = pd.merge(docs_df, word_counts_df, how="inner", on="dict_key", suffixes=["", "_right"])
docs_df = merged_df[["文書ID", "単語ID", "表層", "品詞", "カテゴリー", "エリア", "dict_key"]]
print(docs_df.shape)
display(docs_df)

### 1.5 ワードクラウド

In [ ]:
draw_wordcloud(word_counts_df)

### 1.6 文書-単語表の作成

In [ ]:
cross_df = pd.crosstab([docs_df['カテゴリー'], docs_df['エリア'], docs_df['文書ID']], docs_df['単語ID'], margins=False)
cross_df.columns = word_counts_df["表層"]
print(cross_df.shape)
display(cross_df)

### 1.7 共起ネットワーク図

#### 1.7.1 共起度行列を作成する

In [ ]:
from scipy.sparse import csc_matrix

X = csc_matrix(cross_df.values)
X[X > 0] = 1
Xc = (X.T * X)
Xc = np.triu(Xc.toarray())

cooccur_df = pd.DataFrame(Xc, columns=cross_df.columns, index=cross_df.columns)
print(cooccur_df.shape)
display(cooccur_df.head())

#### 1.7.2 Jaccard 係数を求める

In [ ]:
jaccard_df = jaccard_coef(cooccur_df)
print(jaccard_df.shape)
display(jaccard_df.head())

#### 1.7.3 プロットする

In [ ]:
word_counts = cross_df.sum(axis=0).values
plot_cooccur_network(jaccard_df, word_counts, np.sort(jaccard_df.values.reshape(-1))[::-1][60])

### 1.8 係り受けネットワーク図 

#### 1.8.1 係り受け行列を作成する

In [ ]:
# チャンク(chunk)から単語を取り出す
def get_words(tree, from_chunk, stopwords):
    beg = from_chunk.token_pos
    end = from_chunk.token_pos + from_chunk.token_size

    words = []
    for i in range(beg, end):
        token = tree.token(i)
        features = token.feature.split(',')
        pos1 = features[0]
        pos2 = features[1] if len(features) > 1 else ""
        base = features[6] if len(features) > 6 else None

        if base not in stopwords:

            if (pos1 == "名詞" and pos2 == "一般"):
                base = base if base is not None else node.surface
                postag = "名詞"
                key = (base, postag)
                words.append(key)

            elif (pos1 == "名詞" and pos2 == "形容動詞語幹"):
                base = base if base is not None else node.surface
                base = f"{base}だ"
                postag = "形容動詞"
                key = (base, postag)
                words.append(key)

            elif pos1 == "形容詞":
                base = base if base is not None else node.surface
                postag = "形容詞"
                key = (base, postag)
                words.append(key)

            elif pos1 == "未知語":
                base = base if base is not None else node.surface
                postag = "未知語"
                key = (base, postag)
                words.append(key)

    return words


import CaboCha

cp = CaboCha.Parser("-r ../tools/usr/local/etc/cabocharc")

ZEN = "".join(chr(0xff01 + i) for i in range(94))
HAN = "".join(chr(0x21 + i) for i in range(94))
HAN2ZEN = str.maketrans(HAN, ZEN)

# stopwords = ['する', 'ある', 'ない', 'いう', 'もの', 'こと', 'よう', 'なる', 'ほう']
stopwords = ['*']

pair_counts = defaultdict(lambda: 1)
pairs = []

for index, row in df.iterrows():
    # print(cp.parseToString(row["コメント"].translate(HAN2ZEN)))
    tree = cp.parse(row["コメント"].translate(HAN2ZEN))

    # chunks = get_chunks(tree)
    chunks = {}
    key = 0
    for i in range(tree.size()):
        tok = tree.token(i)
        if tok.chunk:
            chunks[key] = tok.chunk
            key += 1

    for from_chunk in chunks.values():
        if from_chunk.link < 0:
            continue
        to_chunk = chunks[from_chunk.link]

        # from_surface = get_surface(tree, from_chunk)
        from_words = get_words(tree, from_chunk, stopwords)

        # to_surface = get_surface(tree, to_chunk)
        to_words = get_words(tree, to_chunk, stopwords)

    for f in from_words:
        for t in to_words:
            key = (f[0], t[0])
            pair_counts[key] += 1

Xc = cooccur_df.values
Xd = np.zeros(Xc.shape)

for (f,t), v in pair_counts.items():
    columns = list(cooccur_df.columns)
    if f in columns and t in columns:
        i = columns.index(f)
        j = columns.index(t)
        Xd[i,j] = v

dependency_df = pd.DataFrame(Xd, columns=cooccur_df.columns, index=cooccur_df.columns)
print(dependency_df.shape)
display(dependency_df.head())

#### 1.8.2 条件付き確率を求める

In [ ]:
Xc = cooccur_df.values
Xd = np.zeros(Xc.shape)
word_counts = cross_df.sum(axis=0).values

for (f,t), v in pair_counts.items():
    columns = list(cooccur_df.columns)
    if f in columns and t in columns:
        i = columns.index(f)
        j = columns.index(t)
        Xd[i,j] = v / word_counts[i]

dependency_df = pd.DataFrame(Xd, columns=cooccur_df.columns, index=cooccur_df.columns)
print(dependency_df.shape)
display(dependency_df.head())

#### 1.8.3 プロットする

In [ ]:
word_counts = cross_df.sum(axis=0).values
plot_dependency_network(dependency_df, word_counts, np.sort(dependency_df.values.reshape(-1))[::-1][60])

### 1.9 対応分析

In [ ]:
print(cross_df.shape)
display(cross_df.head())

#### 1.9.1 クロス集計表を作成する

In [ ]:
category_cross_df = cross_df.groupby(level='カテゴリー').sum()
area_cross_df = cross_df.groupby(level='エリア').sum()
aggregate_df = pd.concat([category_cross_df, area_cross_df])
print(aggregate_df.shape)
display(aggregate_df)

#### 1.9.2 対応分析プロットを作成する

In [ ]:
plot_coresp(aggregate_df)

### 1.10 トピックモデル

In [ ]:
print(cross_df.shape)
display(cross_df.head())

#### 1.10.1 トピックを抽出する (LDA)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(max_iter=25, learning_method='batch', random_state=0, n_jobs=-1, n_components=6)
lda.fit(cross_df.values)

n_top_words = 20
feature_names = cross_df.columns

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic # {topic_idx+1}:", end=" ")
    for i in topic.argsort()[:-n_top_words-1:-1]:
        print(feature_names[i], end=" ")
    print()

#### 1.10.2 結果をワードクラウドで描画する

In [ ]:
n_top_words = 100
draw_topic_model(lda, feature_names, n_top_words)

## 2. エリアごとで頻出単語を比較する

### 2.1 レジャー

#### 2.1.1 単語の出現頻度

In [ ]:
docs_A_df = docs_df[docs_df['カテゴリー'] == "A_レジャー"]
word_counts_A_df = pd.DataFrame(docs_A_df["単語ID"].value_counts()).reset_index()
word_counts_A_df.columns = ["単語ID", "出現頻度"]
merged_df = pd.merge(word_counts_A_df, word_counts_df, how="inner", on="単語ID", suffixes=["", "_right"])
word_counts_A_df = merged_df[["単語ID", "表層", "出現頻度", "dict_key"]][0:75]
print(word_counts_A_df.shape)
display(word_counts_A_df.head(10))

In [ ]:
merged_df = pd.merge(docs_A_df, word_counts_A_df, how="inner", on="dict_key", suffixes=["", "_right"])
docs_A_df = merged_df[["文書ID", "単語ID", "表層", "品詞", "カテゴリー", "エリア", "dict_key"]]
print(docs_A_df.shape)
display(docs_A_df)

#### 2.1.2 ワードクラウド

In [ ]:
draw_wordcloud(word_counts_A_df)

### 2.2 ビジネス

#### 2.2.1 単語の出現頻度

In [ ]:
docs_B_df = docs_df[docs_df['カテゴリー'] == "B_ビジネス"]
word_counts_B_df = pd.DataFrame(docs_B_df["単語ID"].value_counts()).reset_index()
word_counts_B_df.columns = ["単語ID", "出現頻度"]
merged_df = pd.merge(word_counts_B_df, word_counts_df, how="inner", on="単語ID", suffixes=["", "_right"])
word_counts_B_df = merged_df[["単語ID", "表層", "出現頻度", "dict_key"]][0:75]
print(word_counts_B_df.shape)
display(word_counts_B_df.head(10))

In [ ]:
merged_df = pd.merge(docs_B_df, word_counts_B_df, how="inner", on="dict_key", suffixes=["", "_right"])
docs_B_df = merged_df[["文書ID", "単語ID", "表層", "品詞", "カテゴリー", "エリア", "dict_key"]]
print(docs_B_df.shape)
display(docs_B_df)

#### 2.1.2 ワードクラウド

In [ ]:
draw_wordcloud(word_counts_B_df)